In [12]:
#to use the language model, make sure you've unzipped the languageModel.tar.gz file
#and have compiled the code in the LanguageModelDecoder folder
baseDir = '/oak/stanford/groups/shenoy/fwillett/speechPaperRelease_final'

In [22]:
import os
from glob import glob
from pathlib import Path
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]=""

import numpy as np
from omegaconf import OmegaConf
import tensorflow as tf
from neuralDecoder.neuralSequenceDecoder import NeuralSequenceDecoder
import neuralDecoder.utils.lmDecoderUtils as lmDecoderUtils

In [6]:
#loads the language model, could take a while and requires ~60 GB of memory
lmDir = baseDir+'/languageModel'
ngramDecoder = lmDecoderUtils.build_lm_decoder(
    lmDir,
    acoustic_scale=0.8, #1.2
    nbest=1,
    beam=18
)

I0611 03:28:30.892040 28987 brain_speech_decoder.h:52] Reading fst /oak/stanford/groups/shenoy/fwillett/speechPaperRelease_final/languageModel/TLG.fst
I0611 03:30:51.773838 28987 brain_speech_decoder.h:81] Reading symbol table /oak/stanford/groups/shenoy/fwillett/speechPaperRelease_final/languageModel/words.txt


In [15]:
#evaluate the RNN on the test partition and competitionHoldOut partition
testDirs = ['test','competitionHoldOut']
trueTranscriptions = [[],[]]
decodedTranscriptions = [[],[]]
for dirIdx in range(2):
    ckptDir = baseDir + '/derived/rnns/baselineRelease'

    args = OmegaConf.load(os.path.join(ckptDir, 'args.yaml'))
    args['loadDir'] = ckptDir
    args['mode'] = 'infer'
    args['loadCheckpointIdx'] = None

    for x in range(len(args['dataset']['datasetProbabilityVal'])):
        args['dataset']['datasetProbabilityVal'][x] = 0.0

    for sessIdx in range(4,19):
        args['dataset']['datasetProbabilityVal'][sessIdx] = 1.0
        args['dataset']['dataDir'][sessIdx] = baseDir+'/derived/tfRecords'
    args['testDir'] = testDirs[dirIdx]

    # Initialize model
    tf.compat.v1.reset_default_graph()
    nsd = NeuralSequenceDecoder(args)

    # Inference
    out = nsd.inference()
    decoder_out = lmDecoderUtils.cer_with_lm_decoder(ngramDecoder, out, outputType='speech_sil', blankPenalty=np.log(2))

    def _ascii_to_text(text):
        endIdx = np.argwhere(text==0)
        return ''.join([chr(char) for char in text[0:endIdx[0,0]]])

    for x in range(out['transcriptions'].shape[0]):
        trueTranscriptions[dirIdx].append(_ascii_to_text(out['transcriptions'][x,:]))  
    decodedTranscriptions[dirIdx] = decoder_out['decoded_transcripts']


2023-06-11 03:36:19.221505: E tensorflow/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-06-11 03:36:19.221555: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: sh03-11n03.int
2023-06-11 03:36:19.221562: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: sh03-11n03.int
2023-06-11 03:36:19.221613: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 530.30.2
2023-06-11 03:36:19.221656: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 530.30.2
2023-06-11 03:36:19.221661: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 530.30.2
2023-06-11 03:36:19.221894: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU inst

Model: "gru"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_1 (GRU)                 multiple                  28317696  
                                                                 
 gru_2 (GRU)                 multiple                  6297600   
                                                                 
 gru_3 (GRU)                 multiple                  6297600   
                                                                 
 gru_4 (GRU)                 multiple                  6297600   
                                                                 
 gru_5 (GRU)                 multiple                  6297600   
                                                                 
 dense (Dense)               multiple                  42025     
                                                                 
Total params: 53,551,145
Trainable params: 53,551,145
Non-train

Trainable params: 65,792
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, None, 256)         65792     
                                                                 
 dropout_4 (Dropout)         (None, None, 256)         0         
                                                                 
Total params: 65,792
Trainable params: 65,792
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, None, 256)         65792     
                                                                 
 dropout_5 (Dropout)         (None, None, 256

Total params: 65,792
Trainable params: 65,792
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_17 (Dense)            (None, None, 256)         65792     
                                                                 
 dropout_16 (Dropout)        (None, None, 256)         0         
                                                                 
Total params: 65,792
Trainable params: 65,792
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_18 (Dense)            (None, None, 256)         65792     
                                                                 
 dropout_17 (Dropout) 

  0%|          | 0/600 [00:00<?, ?it/s]

Model: "gru"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_1 (GRU)                 multiple                  28317696  
                                                                 
 gru_2 (GRU)                 multiple                  6297600   
                                                                 
 gru_3 (GRU)                 multiple                  6297600   
                                                                 
 gru_4 (GRU)                 multiple                  6297600   
                                                                 
 gru_5 (GRU)                 multiple                  6297600   
                                                                 
 dense (Dense)               multiple                  42025     
                                                                 
Total params: 53,551,145
Trainable params: 53,551,145
Non-train

 dropout_3 (Dropout)         (None, None, 256)         0         
                                                                 
Total params: 65,792
Trainable params: 65,792
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, None, 256)         65792     
                                                                 
 dropout_4 (Dropout)         (None, None, 256)         0         
                                                                 
Total params: 65,792
Trainable params: 65,792
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)        

 dropout_15 (Dropout)        (None, None, 256)         0         
                                                                 
Total params: 65,792
Trainable params: 65,792
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_17 (Dense)            (None, None, 256)         65792     
                                                                 
 dropout_16 (Dropout)        (None, None, 256)         0         
                                                                 
Total params: 65,792
Trainable params: 65,792
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_18 (Dense)     

  0%|          | 0/1200 [00:00<?, ?it/s]

In [19]:
from neuralDecoder.utils.lmDecoderUtils import _cer_and_wer as cer_and_wer

#get word error rate and phoneme error rate for the test set (cer is actually phoneme error rate here)
cer, wer = cer_and_wer(decodedTranscriptions[0], trueTranscriptions[0], outputType='speech_sil', returnCI=True)

#print word error rate
print(wer)

(0.1854970124932102, 0.1700017381015154, 0.20180266599821478)


In [17]:
#print the sentence predictions for the test set
print(decodedTranscriptions[0])

["it's really hard to find something that works", 'and i love the shining', 'i do have a friend that one', 'the way in the back', 'actually it makes sense to a certain extent', 'so the government that he was employed', "they feel good about what they're doing", 'beside it an invasion of personal privacy', 'so you travel a lot', 'they were going to write', 'in kind of a good environment to live in', 'the best of android', "i don't know what you think about the point", 'i believe it was about ten dollars', 'what in the world talk are you on to', 'android tv box', 'a couple of times', 'i had the back down on it', 'what i do as a rift and has been real handy', 'i do when i was working', "it's a package deal", 'i have seen it', 'the measure already passed a senate committee', 'they told me that that was a topic', 'in the red cross or weaver said', 'they did a lot of play work and stuff', 'you can can too much right', 'recent legislation', "i'm a car buff too", 'two things from a grateful he

In [18]:
#print the predictions for the competition hold-out set (labels are unreleased)
print(decodedTranscriptions[1])

["i'm originally from colorado", "if he had money i'd be pay to buy a new", 'wasted in york', 'i thought the topic was boring', 'but every life up there', "you don't get involve", 'hope you enjoy', "we don't do enough", 'a voice from the crowd', 'i hope you enjoy my back', 'i never have it', 'maybe even too far', "it's sad that a nation we got some time", 'to invest or suit or dress', 'manufactured by nineteen seventy five', 'the water takes the resistance', "because that's all she talked about", 'did you hear an piece about that', 'i play in devil company workers', "i've heard on the news", 'maybe put in a little bit of it', 'and i guess it was that me', 'in just cause', 'the word of the confusion', 'and it evoked an emotional response', 'you want to buy something', "that's basically what we do", 'i guess a lot of people do', 'a lot of people have real water', 'it seems like he would develop ford', 'will you not', 'they make a collection on money for yard waste', 'several of the count